# Decision Tree Classifier from Scratch
***
## Table of Contents
***

In [228]:
from sklearn import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Decision trees and regression trees are collectively referred to as **CART**, which stands for **Classification and Regression Trees**.

- **Decision Trees** are used for classification tasks, where the target variable is *categorical*.

- **Regression Trees** are used for regression tasks, where the target variable is *continuous*.

CART is a popular algorithm that can handle both types of tasks by optimising for different criteria:

- For classification, CART minimises classification error, Gini impurity, or entropy.

- For regression, CART minimises the mean squared error (MSE) or mean absolute error (MAE).

Both types of trees follow the same core idea of splitting the data based on conditions to create homogeneous subsets, but their objectives differ depending on the problem type.
In this notebook, we will build a predictive model using Decision Trees on the breast cancer dataset from the scikit-learn library.

## 1. Load Data

In [229]:
# Load the dataset
data = datasets.load_breast_cancer()
# data = datasets.load_iris()
X, y = data.data, data.target
feature_names = data.feature_names

# Check the shape of the data
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Features: \n{feature_names}")

Features shape: (569, 30)
Target shape: (569,)
Features: 
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [ ]:
df = pd.DataFrame(data=X, columns=feature_names)
df['diagnosis'] = y
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [252]:
df['target'].value_counts()

target
1    357
0    212
Name: count, dtype: int64

In this dataset, the features represent the characteristics of breast cancer (e.g., radius, texture, etc.), while the target is a boolean value indicating whether the tumour is benign (0) or malignant (1).

## 2. Train Test Split
Train test split is a fundamental model validation technique in machine learning. It divides a dataset into two separate portions: a **training set** used to train a model, and a **testing set** used to evaluate how well the model can perform on unseen data. 

The typical split ratio is 80% for training and 20% for testing, though this can vary (70/30 or 90/10 are also common). The key principle is that the test set must remain completely separated during model training process, and should never be used to make decisions about the model or tune parameters. 

The split is usually done randomly to ensure both sets are representative of the overall dataset, and many libraries (such as scikit-learn) provide build-in functions that handle this process automatically while maintaining proper randomisation.


In [230]:
def train_test_split(X: np.array, y: np.array, test_size: float = 0.2,
                     random_state: int = None) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Split arrays or matrices into random train and test subsets.

    Parameters:
        X (np.array): Input features, a 2D array with rows (samples) and columns (features).
        y (np.array): Target values/labels, a 1D array with rows (samples).
        test_size (float): Proportion of the dataset to include in the test split. Must be between 0.0 and 1.0. default = 0.2
        random_state (int): Seed for the random number generator to ensure reproducible results. default = None

    Returns:
        tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        A tuple containing:
            - X_train (np.ndarray): Training set features.
            - X_test (np.ndarray): Testing set features.
            - y_train (np.ndarray): Training set target values.
            - y_test (np.ndarray): Testing set target values.
    """
    # Set a random seed if it exists
    if random_state:
        np.random.seed(random_state)

    # Create a list of numbers from 0 to len(X)
    indices = np.arange(len(X))

    # Shuffle the indices
    np.random.shuffle(indices)

    # Define the size of our test data from len(X)
    test_size = int(test_size * len(X))

    # Generate indices for test and train data
    test_indices: list[int] = indices[:test_size]
    train_indices: list[int] = indices[test_size:]

    # Return: X_train, X_test, y_train, y_test
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

## 3. Gini Impurity and Entropy

In [231]:
def gini(y):
    proportions = np.bincount(y) / len(y)
    return 1 - np.sum(proportions**2)

def entropy(y):
    proportions = np.bincount(y) / len(y)
    proportions = proportions[proportions > 0]  # Avoid log(0)
    return -np.sum(proportions * np.log2(proportions))

In [232]:
print("Gini Impurity:", gini(y))
print("Entropy:", entropy(y))

Gini Impurity: 0.4675300607546925
Entropy: 0.9526351224018599


## 4. Information Gain
The information_gain function calculates the difference between the metric for the parent node and the weighted average of the metrics for the child nodes (left and right splits).

In [233]:
def information_gain(y, y_left, y_right, metric="gini"):
    """
    Calculate the information gain for a split.
    Args:
    - y: Target values of the parent node.
    - y_left: Target values of the left child node.
    - y_right: Target values of the right child node.
    - metric: The metric to use ('gini' or 'entropy').
    
    Returns:
    - Information gain (float).
    """
    if metric == "gini":
        parent_metric = gini(y)
        left_metric = gini(y_left)
        right_metric = gini(y_right)
    else:  # metric == "entropy"
        parent_metric = entropy(y)
        left_metric = entropy(y_left)
        right_metric = entropy(y_right)

    weighted_metric = (
        len(y_left) / len(y) * left_metric
        + len(y_right) / len(y) * right_metric
    )
    return parent_metric - weighted_metric


## 5. Find the best split
This function identifies the best feature and threshold to split the data using the specified metric (Gini or Entropy).
Steps:

1. Loop through all features.

2. For each feature, iterate over all unique thresholds.

3. Split the data into left and right subsets based on the threshold.

4. Compute the Gini/Entropy for both subsets and calculate the weighted average.

5. Keep track of the split with the lowest metric value.

In [234]:
def best_split(X, y, metric="gini", feature_names=None):
    best_info_gain = float("-inf")
    best_split = None
    n_features = X.shape[1]

    for feature in range(n_features):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            left_mask = X[:, feature] <= threshold
            right_mask = X[:, feature] > threshold

            if sum(left_mask) == 0 or sum(right_mask) == 0:
                continue

            info_gain = information_gain(y, y[left_mask], y[right_mask], metric)

            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_split = {
                    "feature_index": feature,
                    "feature_name": feature_names[feature] if feature_names is not None else feature,
                    "threshold": threshold,
                }

    return best_split

In [235]:
split = best_split(X, y, metric="gini", feature_names=feature_names)
print("Best Split:", split)

Best Split: {'feature_index': 20, 'feature_name': np.str_('worst radius'), 'threshold': np.float64(16.77)}


## 6. Build Tree
This function recursively builds the decision tree.

Steps:

1. Stop the recursion if all labels are identical or the maximum depth is reached.

2. Find the best split using best_split.

3. Split the data into left and right subsets.

4. Recursively build the left and right subtrees.

5. Return the tree structure as a nested dictionary.

In [236]:
def build_tree(X, y, max_depth=None, depth=0, metric="gini", feature_names=None):
    if len(set(y)) == 1 or (max_depth is not None and depth == max_depth):
        return {"type": "leaf", "value": np.argmax(np.bincount(y))}

    split = best_split(X, y, metric, feature_names)
    if not split:
        return {"type": "leaf", "value": np.argmax(np.bincount(y))}

    # Use feature_index for calculations
    left_mask = X[:, split["feature_index"]] <= split["threshold"]
    right_mask = X[:, split["feature_index"]] > split["threshold"]

    left_tree = build_tree(X[left_mask], y[left_mask], max_depth, depth + 1, metric, feature_names)
    right_tree = build_tree(X[right_mask], y[right_mask], max_depth, depth + 1, metric, feature_names)

    return {
        "type": "node",
        "feature": split["feature_name"],
        "threshold": split["threshold"],
        "left": left_tree,
        "right": right_tree,
    }

## 7. Traverse the Tree For Prediction
This function traverses the tree to make predictions for a single sample.

In [237]:
def traverse_tree(x, tree, feature_names=None):
    """
    Traverse the decision tree to make a prediction for a single sample.
    Args:
    - x: Single sample (1D array).
    - tree: Decision tree structure.
    - feature_names: List of feature names (optional, needed for name-to-index mapping).
    
    Returns:
    - Predicted label (int).
    """
    if tree["type"] == "leaf":
        return tree["value"]

    # Resolve feature index if feature_names is provided
    feature_index = feature_names.index(tree["feature"]) if feature_names is not None else tree["feature"]

    if x[feature_index] <= tree["threshold"]:
        return traverse_tree(x, tree["left"], feature_names)
    else:
        return traverse_tree(x, tree["right"], feature_names)



In [238]:
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

## 8. Prediction
This function predicts labels for all samples in the dataset.

In [239]:
def predict(X, tree, feature_names=None):
    """
    Predict the label(s) for the given data.
    Args:
    - X: Input features (2D array or 1D array for single sample).
    - tree: Decision tree structure.
    - feature_names: List of feature names (optional).
    
    Returns:
    - Predicted labels (1D array or single label).
    """
    if len(X.shape) == 1:  # If a single sample is provided
        return traverse_tree(X, tree, feature_names)
    return np.array([traverse_tree(x, tree, feature_names) for x in X])



In [240]:
# Load the dataset
# data = datasets.load_iris()
data = datasets.load_breast_cancer()
X, y = data.data, data.target
feature_names = data.feature_names.tolist()

# Build the tree using Gini impurity
tree_gini = build_tree(X, y, max_depth=3, metric="gini", feature_names=feature_names)

# Display the tree
import pprint
pprint.pprint(tree_gini)

{'feature': 'worst radius',
 'left': {'feature': 'worst concave points',
          'left': {'feature': 'radius error',
                   'left': {'type': 'leaf', 'value': np.int64(1)},
                   'right': {'type': 'leaf', 'value': np.int64(0)},
                   'threshold': np.float64(0.8811),
                   'type': 'node'},
          'right': {'feature': 'worst texture',
                    'left': {'type': 'leaf', 'value': np.int64(1)},
                    'right': {'type': 'leaf', 'value': np.int64(0)},
                    'threshold': np.float64(25.5),
                    'type': 'node'},
          'threshold': np.float64(0.1357),
          'type': 'node'},
 'right': {'feature': 'mean texture',
           'left': {'feature': 'mean concave points',
                    'left': {'type': 'leaf', 'value': np.int64(1)},
                    'right': {'type': 'leaf', 'value': np.int64(0)},
                    'threshold': np.float64(0.06211),
                    'type': 'nod

## 9. Evaluate Model

In [241]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build the tree on training data
tree_gini = build_tree(X_train, y_train, max_depth=3, metric="gini", feature_names=feature_names)

# Predict on the test set
y_pred = predict(X_test, tree_gini, feature_names=feature_names)

# Calculate accuracy
test_accuracy = accuracy(y_test, y_pred)
print("Test Accuracy:", test_accuracy)

# Predict a single sample
single_sample = X_test[0]
single_prediction = predict(single_sample, tree_gini, feature_names=feature_names)
print("Predicted label for single sample:", single_prediction)
print("Actual label for single sample:", y_test[0])


Test Accuracy: 0.9647058823529412
Predicted label for single sample: 1
Actual label for single sample: 1
